In [43]:
!pip install -q googlemaps

In [44]:
import json

secrets = {
    'ptv_id': 'your ptv id',
    'ptv_key': 'your ptv key',
    'gmaps_key': 'your google maps key'}

with open('secrets_json', 'w') as json_file:
    json.dump(secrets, json_file)

In [45]:
import json
import pprint
import googlemaps
from ptvv3 import PTVv3
from utils import setup_ptv, setup_gmaps

with open('secrets.json', 'r') as json_file:
    secrets = json.load(json_file)

ptv = setup_ptv(PTVv3(secrets['ptv_id'], secrets['ptv_key']))
assert ptv('/v3/route_types')

gmaps = setup_gmaps(googlemaps.Client(secrets['gmaps_key']))
assert gmaps.reverse_geocode((40.714224, -73.961452))

## Find bus routes and closest stops around a GPS coordinate
We use Google Maps walking directions to compute actual distance. The one returned from the PTV API is the shortest path *as the crow flies*.

In [18]:
from copy import deepcopy

buses = {
    'oakleigh': {'gps': (-37.9057, 145.0927), 'walking_radius': 15},
    'huntingdale': {'gps': (-37.911118, 145.102859), 'walking_radius': 3},
    'monash': {'gps': (-37.910389, 145.129639), 'walking_radius': 15}}

trains = deepcopy(buses)

In [19]:
from utils import find_routes_and_stops, filter_by_walking_distance

def discover(locations, radius=1500, route_types=2):
    for location_name, location_data in locations.items():
        find_routes_and_stops(location_data, radius=radius, route_types=route_types)
        if 22833 in location_data['stops']:  # returned GPS is for other side of the road, confounding walking directions
            location_data['stops'][22833]['gps'] = (-37.90536, 145.09135)  # would otherwise say 5 minutes instead of the 2
        filter_by_walking_distance(location_data)
        print(f'Found {len(location_data["routes"]):2} routes/directions from {len(location_data["stops"]):2} stops around {location_name.capitalize()}')

In [20]:
discover(buses)  # updates in place

Found 14 routes/directions from 12 stops around Oakleigh
Found  4 routes/directions from  1 stops around Huntingdale
Found 13 routes/directions from  4 stops around Monash


## Stops and routes example

In [21]:
print('Stops')
for index, stop_data in enumerate(sorted(buses['oakleigh']['stops'].values(), key=lambda s: s['id'])):
    routes = sorted(buses['oakleigh']['routes'][route_id]['number'] for route_id in stop_data['routes'])
    print(f'{index+1:2}  {stop_data["id"]}: {", ".join(routes)}')

print('\nRoutes')
for index, route_data in enumerate(sorted(buses['oakleigh']['routes'].values(), key=lambda r: r['number'])):
    stops = sorted(direction['stop'] for direction in route_data['directions'].values())
    print(f'{index+1:2}  {route_data["number"]}: {stops}')

Stops
 1  10004: 624, 693
 2  10005: 800, 802, 804, 862
 3  13989: 625, 701
 4  15779: 693
 5  16339: 704, 742, 900
 6  18098: 630
 7  19029: 701
 8  22833: 733
 9  22875: 630
10  25468: 903
11  25746: 903
12  27477: 624

Routes
 1  624: [10004, 27477]
 2  625: [13989, 13989]
 3  630: [18098, 22875]
 4  693: [10004, 15779]
 5  701: [13989, 19029]
 6  704: [16339, 16339]
 7  733: [22833, 22833]
 8  742: [16339, 16339]
 9  800: [10005]
10  802: [10005]
11  804: [10005]
12  862: [10005]
13  900: [16339, 16339]
14  903: [25468, 25746]


In [22]:
route_id, stop_id = 13135, 13989

print(f"Route #{route_id} (bus {buses['oakleigh']['routes'][route_id]['number']})")
pprint.pprint(buses['oakleigh']['routes'][13135])  # note: different closest stop for different direction

print(f"\nStop #{stop_id} ({buses['oakleigh']['stops'][stop_id]['name']})")
pprint.pprint(buses['oakleigh']['stops'][13989])  # note: same stop for both directions

Route #13135 (bus 701)
{'directions': {184: {'name': 'Oakleigh', 'sequence': 38, 'stop': 19029},
                207: {'name': 'Bentleigh', 'sequence': 1, 'stop': 13989}},
 'id': 13135,
 'name': 'Oakleigh - Bentleigh via Mackie Road & Brady Road',
 'number': '701',
 'type': 2}

Stop #13989 (Oakleigh Station/Johnson St)
{'address': '15 Johnson St, Oakleigh VIC 3166, Australia',
 'distance': 737.651367,
 'gps': (-37.9005356, 145.087433),
 'id': 13989,
 'name': 'Oakleigh Station/Johnson St',
 'routes': {13027: {27: 3, 181: 46}, 13135: {207: 1}},
 'walking': {'distance': {'text': '0.9 km', 'value': 904},
             'duration': {'text': '12 mins', 'value': 746}}}


## Find connecting routes between two locations
Given the bus stops previously discovered, find routes in common between two locations. Only direct connections are considered.

The correct direction is identified comparing the sequence number of the stops along the route.

In [23]:
from utils import find_connecting_routes, compute_travel_time

oakleigh_to_monash = find_connecting_routes(buses['oakleigh'], buses['monash'])
compute_travel_time(oakleigh_to_monash);  # add travel times

In [164]:
from IPython.display import HTML, display

def format_time(seconds):
    return f'{seconds // 60:2}:{seconds % 60:02}' if seconds is not None else 'N/A'

def display_connections(connections, from_location, to_location):
    headers = '# Route Min._time Avg._time Max_time Walk_to Walk_at From To'
    header = ''.join(f'<th>{name.replace("_", " ")}</th>' for name in headers.split())
    contents = ''.join(f'<tr>{row}</tr>' for row in (
        ''.join(f'<td>{cell}</td>' for cell in (
            index + 1,
            connection["number"],
            format_time(connection["duration"]["min"]),
            format_time(connection["duration"]["avg"]),
            format_time(connection["duration"]["max"]),
            format_time(connection["walking"][connection["forward_direction"]["origin"]]),
            format_time(connection["walking"][connection["forward_direction"]["destination"]]),
            from_location["stops"][connection["forward_direction"]["origin"]]["name"],
            to_location["stops"][connection["forward_direction"]["destination"]]["name"]))
        for index, connection in enumerate(connections.values())))
    
    display(HTML(f'<table><thead><tr>{header}</tr></thead>{contents}</table>'))

display_connections(oakleigh_to_monash, buses["oakleigh"], buses["monash"])

#,Route,Min. time,Avg. time,Max time,Walk to,Walk at,From,To
1,630,8:00,10:56,16:00,9:31,7:10,Best St/North Rd,Monash University
2,733,20:00,24:51,29:00,1:55,7:10,Willgilson Ct/Golf Links Ave,Monash University
3,742,14:00,15:51,18:00,12:20,8:39,Westminster St/Burlington St,Monash University/Research Way
4,800,11:00,12:52,17:00,9:48,9:13,Oakleigh SC/Hanover St,Wellington Rd/Princes Hwy
5,802,11:00,13:00,16:00,9:48,7:10,Oakleigh SC/Hanover St,Monash University
6,804,10:00,13:22,17:00,9:48,7:10,Oakleigh SC/Hanover St,Monash University
7,862,10:00,12:11,16:00,9:48,7:10,Oakleigh SC/Hanover St,Monash University
8,900,10:00,14:39,19:00,12:20,7:10,Westminster St/Burlington St,Monash University


## Example of connection entry

In [165]:
oakleigh_to_monash[13067]

{'id': 13067,
 'type': 2,
 'name': 'Elwood - Monash University via Gardenvale & Ormond & Huntingdale',
 'number': '630',
 'forward_direction': {'id': 188, 'origin': 22875, 'destination': 33430},
 'reverse_direction': {'id': 189, 'origin': 33430, 'destination': 18098},
 'walking': {22875: 571, 33430: 430, 18098: 617},
 'duration': {'min': 480, 'max': 960, 'avg': 656}}

## Add entry for train

In [175]:
discover(trains, radius=1500, route_types=0)

oakleigh_to_huntingdale = find_connecting_routes(trains['oakleigh'], trains['huntingdale'])
compute_travel_time(oakleigh_to_huntingdale)

huntingdale_to_monash = find_connecting_routes(buses['huntingdale'], buses['monash'])
display_connections(compute_travel_time(huntingdale_to_monash), buses['huntingdale'], buses['monash'])

for connection in oakleigh_to_huntingdale.values():
    connection['number'] = 'Train'  # connection['name']
    bus_to_monash = connection['walking'][connection['forward_direction']['destination']] + 465 + 120  # 7:45 via 601/630/900 + 2min buffer
    connection['walking'][connection['forward_direction']['destination']] = 430  # 7:10 walk from Monash bus station
    connection['duration'] = {k: v + bus_to_monash for k, v in connection['duration'].items()}

display_connections(oakleigh_to_huntingdale, trains['oakleigh'], trains['huntingdale'])
merged_connections = oakleigh_to_monash | oakleigh_to_huntingdale

Found  2 routes/directions from  1 stops around Oakleigh
Found  2 routes/directions from  1 stops around Huntingdale
Found  0 routes/directions from  0 stops around Monash


#,Route,Min. time,Avg. time,Max time,Walk to,Walk at,From,To
1,601,N/A,N/A,N/A,0:20,7:10,Huntingdale Station/Haughton Rd,Monash University
2,630,6:00,7:49,10:00,0:20,7:10,Huntingdale Station/Haughton Rd,Monash University
3,900,5:00,7:40,10:00,0:20,7:10,Huntingdale Station/Haughton Rd,Monash University


#,Route,Min. time,Avg. time,Max time,Walk to,Walk at,From,To
1,Train,12:27,12:36,17:27,13:41,7:10,Oakleigh,Huntingdale
2,Train,12:27,12:36,17:27,13:41,7:10,Oakleigh,Huntingdale


## Check routes

In [214]:
from utils import check_routes

def display_services(services, connections):
    """
    TODOs:
    * need to merge rather than substitute the results using the run reference
    * need to remember if a run was seen active in the past, and consider it active
    * change data returned by check routes so that times and delta can be recomputed when merging (or updated live!)
    * re: the above point, make services a field within the connection db instead of a separate structure?
    """
    legend = '⚫🔴🟡🟢'
    headers = 'Status Number Exit_in Arrive_by Alerts'
    header = ''.join(f'<th>{name.replace("_", " ")}</th>' for name in headers.split())
    contents = ''.join(f'<tr>{row}</tr>' for row in (
        ''.join(f'<td title="{tooltip}">{cell}</td>' for cell, tooltip in (
            (legend[sum(service['health'].values())], '\n'.join(('✖✔'[h] + ' ' + n.replace('_', ' ')) for n, h in service['health'].items())),
            (f'{connections[service["route_id"]]["number"]}', f'{connections[service["route_id"]]["name"]}'),
            (print_td(service['get_going_in']), ''),
            # (service['departure_time'].astimezone(tz.tzlocal()).strftime('%H:%M'), ''),
            (service['arrive_by'].astimezone(tz.tzlocal()).strftime('%H:%M'), ''),
            ('❗'*len(service['disruptions']), '\n\n'.join(service["disruptions"]))))
        for service in sorted(services.values(), key=lambda s: s['arrive_by'])))
    
    display(HTML(f'<table><thead><tr>{header}</tr></thead>{contents}</table>'))

# display_services(check_routes(oakleigh_to_monash))
display_services(check_routes(merged_connections), merged_connections)

In [ ]:
import matplotlib.pyplot as plot
import numpy as np

def fix_path(routes):

    plot.figure(figsize=(15, 11))

    coords = locations['oakleigh']['gps']
    plot.plot(coords[1], coords[0], 'r*', markersize=16)  # annotate
    
    coords = locations['monash']['gps']
    plot.plot(coords[1], coords[0], 'r*', markersize=16)
    
    for index, (route_id, route_data) in enumerate(routes.items()):

        if route_data['route']['bus'] == '802':
            route_data['route']['bus'] = '802/804/862'
        
        if route_data['route']['bus'] == '804':
            continue
        
        if route_data['route']['bus'] == '862':
            continue
        
        colors = 'tab:blue tab:orange tab:green tab:red tab:purple tab:brown tab:pink tab:gray tab:olive tab:cyan'.split()
        displacement = [-0.0002, -0.0004, 0.0002, 0.0000, 0.0002, 0.0000, 0.0000, 0.0000][index]
        
        coords = np.array(route_data['path']['coords'])

        if route_data['geopath']:
            for path in route_data['geopath']['coords']:
                geocoords = np.array(path)

                index_start = np.argmin(np.linalg.norm(geocoords - np.array(locations['oakleigh']['gps']), axis=1))
                index_dest = np.argmin(np.linalg.norm(geocoords - np.array(locations['monash']['gps']), axis=1))
                
                geocoords += displacement
                plot.plot(geocoords[index_start:index_dest, 1], geocoords[index_start:index_dest, 0],
                          colors[index % len(colors)], linestyle='-', linewidth=2)

        coords += displacement
        plot.plot(coords[:, 1], coords[:, 0], '.', color=colors[index % len(colors)],
                  markersize=8, label=f"{route_data['route']['bus']} {route_data['geopath']['name']}")
        plot.plot(coords[0, 1], coords[0, 0], '.', color=colors[index % len(colors)], markersize=16)
        plot.plot(coords[-1, 1], coords[-1, 0], '.', color=colors[index % len(colors)], markersize=16)

        # annotate min walking, bus name
    
    plot.axis('equal')
    plot.legend(loc=(0.25, 0.2), prop={'size': 16})
    plot.show()

    # find minimum indexes
    # append corresponding
    # repeat

    # take min in both direction
    # expectation (median) over gaps

fix_path(oakleigh_monash)

In [204]:
from importlib import reload
import utils
utils = reload(utils)

from utils import setup_ptv, setup_gmaps
setup_ptv(ptv), setup_gmaps(gmaps);